# Model Design: Neutal Network creation with TensorFlow

In [41]:
# LOAD LIBS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
tf.config.run_functions_eagerly(True)

from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam, Lion
from tensorflow.keras.regularizers import l1_l2

import warnings
import os

# Ocultar advertencias de TensorFlow y otros warnings innecesarios
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Oculta logs de TensorFlow (0 = todos, 1 = INFO, 2 = WARNING, 3 = ERROR)
warnings.filterwarnings("ignore", category=UserWarning)  # Oculta warnings de Python

# Desactivar ejecución ansiosa (si no la necesitas)
tf.config.run_functions_eagerly(False)

PRICE = 'SalePrice'
# PRICE = 'price'

In [42]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [43]:
# LOAD THE DATASET FROM PREVIOUS CSV
# path = '/home/mike/Escritorio/codes/projects/POLARAI/neural-nets/data_m/csv_files/processed_data.csv'
path = '/home/mike/Escritorio/codes/projects/POLARAI/neural-nets/data_m/csv_files/03_kpi_data.csv'
data = pd.read_csv(path)

# Get number of rows and columns
num_rows, num_cols = data.shape

print(f"Number of records (rows): {num_rows}")
print(f"Number of columns: {num_cols}")

# Optional: Display column names
print("\nColumn names:")
print(data.columns.tolist())

Number of records (rows): 1460
Number of columns: 231

Column names:
['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'ExterQual_TA', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 'KitchenQual_TA', 'YearRemodAdd', 'Foundation_PConc', 'MasVnrArea', 'Fireplaces', 'ExterQual_Gd', 'BsmtQual_TA', 'BsmtFinType1_GLQ', 'Neighborhood_NridgHt', 'BsmtFinSF1', 'SaleType_New', 'SaleCondition_Partial', 'Foundation_CBlock', 'Neighborhood_NoRidge', 'WoodDeckSF', 'KitchenQual_Gd', '2ndFlrSF', 'OpenPorchSF', 'HeatingQC_TA', 'BsmtExposure_Gd', 'Exterior2nd_VinylSd', 'Exterior1st_VinylSd', 'MSZoning_RM', 'HalfBath', 'LotShape_Reg', 'LotArea', 'BsmtExposure_No', 'CentralAir', 'MSZoning_RL', 'HouseStyle_2Story', 'SaleType_WD', 'Electrical_SBrkr', 'RoofStyle_Hip', 'BsmtQual_Gd', 'BsmtFullBath', 'RoofStyle_Gable', 'Neighborhood_StoneBr', 'BsmtUnfSF', 'PavedDrive', 'Neighborhood_OldTown', 'Neighborhood_NAmes', 'Neighborhood_Edwards', 'RoofMatl_WdShngl', 'BedroomAbvGr'

In [44]:
X = data.drop(columns=[PRICE])
y = data[PRICE] 

# First check for and handle infinite values
X = X.replace([np.inf, -np.inf], np.nan)

# Then handle NaN values
X = X.fillna(X.mean())

In [45]:
# NORMALIZE THE DATA (MIN-MAX 1,0)
price_scaler = StandardScaler()
scaler = StandardScaler()

all_scaled_price = price_scaler.fit_transform(y.to_numpy().reshape(-1, 1)) 
all_scaled_vars = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(all_scaled_vars, all_scaled_price, test_size=0.2, random_state=42)

In [46]:
X_train_scaled = X_train
X_test_scaled = X_test

y_train_scaled = y_train
y_test_scaled = y_test

print(X_train_scaled.shape)
print(y_train_scaled.shape)
print(X_test_scaled.shape)
print(y_test_scaled.shape)

(1168, 230)
(1168, 1)
(292, 230)
(292, 1)


In [47]:
# CREATE THE MODEL DEFINITION FUNCTION

def build_model(optimizer_name='SGD'):
    optimizers_dict = {
        'Adam': Adam(learning_rate=0.001),
        'SGD': SGD(learning_rate=0.001, momentum=0.9),
        'RMSprop': RMSprop(learning_rate=0.001),
        'Adagrad': Adagrad(learning_rate=0.001),
        'Adadelta': Adadelta(learning_rate=1.0),
        'Adamax': Adamax(learning_rate=0.002),
        'Nadam': Nadam(learning_rate=0.001),
        'Lion' : Lion(learning_rate=0.001)
    }

    if optimizer_name not in optimizers_dict:
        raise ValueError(f"Optimizador {optimizer_name} no reconocido. Opciones válidas: {list(optimizers_dict.keys())}")

    optimizer = optimizers_dict[optimizer_name]

    model = Sequential([
        layers.Input(shape=(X_train_scaled.shape[1],)),
        layers.Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        layers.Dropout(0.1),  # Dropout aumentado
        layers.Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        layers.Dropout(0.1),  # Dropout aumentado
        layers.Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        layers.Dropout(0.1),  # Dropout en cada capa
        layers.Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        layers.Dropout(0.1),  # Dropout en cada capa
        layers.Dense(1, activation='linear')  # Capa de salida
    ])
    
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

In [48]:
# # K-FOLDING TO SELECT BEST OPTIMIZER

# # Definir validación cruzada
# kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# # Lista de optimizadores a probar
# optimizers = ['Adam', 'SGD', 'Adadelta', 'Lion']

# # Diccionario para almacenar resultados
# results = {}

# early_stopping = EarlyStopping(monitor='val_mae', patience=10, restore_best_weights=True)

# for opt_name in optimizers:
#     fold_mae = []

#     print(f"Entrenando con el optimizador: {opt_name}")

#     for train_idx, val_idx in kfold.split(X_train_scaled):
#         X_train_fold, X_val_fold = X_train_scaled[train_idx], X_train_scaled[val_idx]
#         y_train_fold, y_val_fold = y_train_scaled[train_idx], y_train_scaled[val_idx]

#         try:
#             # Crear un nuevo modelo con el optimizador especificado
#             model = build_model(optimizer_name=opt_name)

#             # Entrenar el modelo
#             history = model.fit(X_train_fold, y_train_fold, epochs=30, batch_size=24,
#                                         validation_data=(X_val_fold, y_val_fold), verbose=0,
#                                         callbacks=[early_stopping])

#             # Evaluar el modelo
#             val_mae = model.evaluate(X_val_fold, y_val_fold, verbose=0)[1]
#             fold_mae.append(val_mae)

#         except ValueError as e:
#             print(f"Error con optimizador {opt_name}: {e}")
#             continue  # Saltar a la siguiente iteración si hay un error

#     # Guardar el MAE promedio en los resultados solo si hubo evaluaciones válidas
#     if fold_mae:
#         results[opt_name] = np.mean(fold_mae)
#     else:
#         results[opt_name] = float('inf')  # Asignar un valor alto si no se pudo entrenar

# print('============================================================')
# # Imprimir los resultados de validación cruzada
# print("\nResultados de Validación Cruzada (MAE promedio):")
# for opt, mae in results.items():
#     print(f"{opt}: {mae:.4f}")
# print('============================================================')
# # Seleccionar el mejor optimizador
# best_optimizer_name = min(results, key=results.get)
# print(f"\nMejor optimizador seleccionado: {best_optimizer_name}")
# print('============================================================')

In [49]:
# CREATE THE MODEL AND
# best_optimizer_name = min(results, key=results.get)
best_optimizer_name = 'Adam'
print(f"Mejor optimizador seleccionado: {best_optimizer_name}")

model = build_model(optimizer_name=best_optimizer_name)

Mejor optimizador seleccionado: Adam


In [50]:
# TRAIN IT

history = model.fit(X_train_scaled, y_train_scaled, epochs=250, batch_size=64,
                    validation_split=0.2, verbose=1, callbacks=[
                        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
                        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1),
                        # Puedes agregar más callbacks como TensorBoard si es necesario
                    ])

Epoch 1/250


15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 194ms/step - loss: 32.4926 - mae: 0.6268 - val_loss: 28.6884 - val_mae: 0.4435 - learning_rate: 0.0010
Epoch 2/250
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 27.7759 - mae: 0.4800 - val_loss: 24.4175 - val_mae: 0.3406 - learning_rate: 0.0010
Epoch 3/250
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 23.5894 - mae: 0.4292 - val_loss: 20.4984 - val_mae: 0.3066 - learning_rate: 0.0010
Epoch 4/250
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 19.7512 - mae: 0.3752 - val_loss: 16.9678 - val_mae: 0.2854 - learning_rate: 0.0010
Epoch 5/250
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 16.2792 - mae: 0.3535 - val_loss: 13.8288 - val_mae: 0.2751 - learning_rate: 0.0010
Epoch 6/250
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 13.2236 - mae: 0.3325 - val_loss: 11.0961 - val_mae: 0.2622 - learning_rate: 0.0010
Epoch 7/250
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10.6636 - mae: 0.3318 - val_loss: 8.8076 - val_mae: 0.2553 - learning_rate: 0.0010
Epoch 8/25

In [52]:
# Evaluar el modelo en el conjunto de prueba
y_pred_scaled = model.predict(X_test_scaled)

# Invertir la escala para obtener las predicciones en la escala original
y_pred = price_scaler.inverse_transform(y_pred_scaled)
y_true = price_scaler.inverse_transform(y_test_scaled)

# Calcular las métricas
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_true, y_pred)

mean_y_true = np.mean(y_true)
rmse_percentage = (rmse / mean_y_true) * 100
mse_percentage = (mse / mean_y_true) * 100

print("============================================================")
print("Métricas del Modelo en el Conjunto de Prueba:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Squared Error (MSE en %): {mse_percentage:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Root Mean Squared Error (RMSE en %): {rmse_percentage:.2f}%")
print(f"R-squared (R2): {r2:.4f}")
print("============================================================")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Métricas del Modelo en el Conjunto de Prueba:
Mean Absolute Error (MAE): 21388.22
Mean Squared Error (MSE): 1213154567.17
Mean Squared Error (MSE en %): 678347.04
Root Mean Squared Error (RMSE): 34830.37
Root Mean Squared Error (RMSE en %): 19.48%
R-squared (R2): 0.8418
